In [ ]:
import pandas as pd
import numpy as np
import datetime
import math
import os

In [ ]:
alldata = pd.read_csv("./data/annotations/alldata.csv")
videos = pd.read_csv("./data/annotations/video_list.csv")
video = videos["Video_File"]
exam = videos["Exam_ID"]
map_ev = dict(zip(exam,video))
alldata["videoid"] = alldata["examid"].apply(lambda x: map_ev[int(x)] if int(x) in map_ev.keys() else None)
vids = os.listdir("./data/openpose/")
vids = [x.split('-')[0] for x in vids]
alldata = alldata.loc[alldata["videoid"].isin(vids)] #only keep data that has a corresponding video available

In [ ]:
#add gdi
gdi = pd.read_csv("data/annotations/gdi.csv")
alldata = alldata.merge(right=gdi,on=['Patient_ID','examid','side'],how='left')

In [ ]:
#only keep videos with data for both the L and R side available
alldata['n_sides'] = alldata.groupby('videoid')['GDI'].transform(len)
alldata = alldata[alldata['n_sides'] == 2]

Per Apoorva's email, I'm counting "a SEMLS as any event where there were at least two surgeries on the analyzed side (i.e., at least two of the upcoming_\*_ips)"

In [ ]:
#compute SEMLS variable
surgInfo = pd.read_csv('data/annotations/surgInfo.csv')
ips_cols = [x for x in list(surgInfo.columns) if x.split('_')[-1] == 'ips']
con_cols = [x for x in list(surgInfo.columns) if x.split('_')[-1] == 'con']
surgInfo['n_ips'] = np.sum(surgInfo[ips_cols],axis=1)
surgInfo['n_con'] = np.sum(surgInfo[con_cols],axis=1)

def is_SEMLS(row):
    if row.n_ips >= 2:
        return 1
    else:
        return 0
surgInfo['SEMLS'] = surgInfo.apply(is_SEMLS,axis=1)

In [ ]:
alldata = alldata.merge(right=surgInfo[['Patient_ID','examid','side','SEMLS']],on=['Patient_ID','examid','side'],how='left')
alldata["SEMLS"] = alldata["SEMLS"].apply(lambda x: 0 if math.isnan(x) else x)

In [ ]:
alldata.to_csv('./data/processed/alldata_processed.csv', index=False)